In [ ]:
import cupy as cp  # CuPy for GPU acceleration
from cupyx.scipy.sparse import csr_matrix

import scipy.sparse as cpx_sparse

import pyvista as pv

import numpy as np  # NumPy for handling some CPU-based operations

from tqdm import tqdm

from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg import gmres
from scipy.sparse.linalg import bicgstab  # Fallback to CPU-based solver
from scipy.ndimage import label

import seaborn as sns

from matplotlib.colors import BoundaryNorm
from matplotlib import cm
import matplotlib.pyplot as plt

In [ ]:
# Flush the CuPy memory cache
cp.get_default_memory_pool().free_all_blocks()

# Check memory pool information
memory_pool = cp.get_default_memory_pool()
print("Allocated memory:", memory_pool.total_bytes() / (1024**3), "GB")

In [ ]:
# Material Properties
cohesion = 0.0  # Cohesion in Pascals
friction_angle = 5  # Friction angle in degrees
density = 3000  # Density in kg/m³
E = 10000  # Young's modulus in Pascals
nu = 0.9  # Poisson's ratio

water_table_depth = 2  # Depth of water table in meters
pore_pressure = 1e4  # Pore pressure in Pascals
surcharge_load = 100000e7  # Surcharge load in Pascals
seismic_coefficient = 0.15  # Seismic load factor

filepath = r"C:\\Users\\alank\\Documents\\Analytics\\Tiny_mesh.vtu"
# ? Convert to class for easy maintenance?

In [ ]:
def load_volume_mesh(filepath):
    """Load the Volume Mesh

    Args:
        filepath (string): filepath to mesh file

    Returns:
        Dataset: mesh as array
    """
    print("Loading volume mesh...")
    print(f"Reading file: {filepath}")
    mesh = pv.read(filepath)
    print("Volume mesh loaded successfully.")
    return mesh


def assign_material_properties(mesh, cohesion, friction_angle, density, E, nu):
    """Assign Material Properties

    Args:
        mesh (pyvista.DataSet): The finite element mesh object that contains the geometry and connectivity information of the entire domain.
        cohesion (int): Cohesion in Pascals
        friction_angle (int): Friction angle in degrees
        density (int): Density in kg/m³
        E (float): Young's modulus in Pascals
        nu (float): Poisson's ratio
    """
    print("Assigning material properties...")
    mesh["cohesion"] = np.full(mesh.n_cells, cohesion)  # Keep it as NumPy
    mesh["friction_angle"] = np.full(mesh.n_cells, friction_angle)  # Keep it as NumPy
    mesh["density"] = np.full(mesh.n_cells, density)  # Keep it as NumPy
    mesh["E"] = np.full(mesh.n_cells, E)  # Keep it as NumPy
    mesh["nu"] = np.full(mesh.n_cells, nu)  # Keep it as NumPy
    print("Material properties assigned.")


# ! Compute Jacobian -> find a native library
def compute_jacobian(element_nodes, dN_dxi):
    """Computes teh jacobian matrix

    Args:
        element_nodes (array): 3x3 matrice
        dN_dxi (array): derivatives of shape functions with respect to natural coordinates

    Returns:
        array: Jacobian
    """
    J = cp.zeros((3, 3))  # 3x3 for a 3D element
    for i in range(4):  # Assuming a 4-node tetrahedral element
        J += cp.outer(dN_dxi[i], element_nodes[i])
    return J


def compute_B_matrix(J_inv, dN_dxi):
    """Strain-Displacement matrix

    Args:
        J_inv (matrix): Inverse jacobian inverse matrix
        dN_dxi (matrx): derivatives of shape functions with respect to natural coordinates

    Returns:
        matrix: Strain-Displacement matrix
    """
    B = cp.zeros((6, 12))  # 6 strains and 3 displacements per node, 4 nodes * 3 = 12
    for i in range(4):
        dN_dx = cp.dot(J_inv, dN_dxi[i])
        B[0, i * 3] = dN_dx[0]  # ε_xx
        B[1, i * 3 + 1] = dN_dx[1]  # ε_yy
        B[2, i * 3 + 2] = dN_dx[2]  # ε_zz
        B[3, i * 3] = dN_dx[1]  # ε_xy
        B[3, i * 3 + 1] = dN_dx[0]
        B[4, i * 3 + 1] = dN_dx[2]  # ε_yz
        B[4, i * 3 + 2] = dN_dx[1]
        B[5, i * 3] = dN_dx[2]  # ε_zx
        B[5, i * 3 + 2] = dN_dx[0]
    return B


def compute_C_matrix(E, nu):
    """Elasticity matrix

    Args:
        E (flaot): Young's modulus
        nu (float): Poisson's ratio

    Returns:
        matrix:  elasticity matrix
    """
    C = cp.zeros((6, 6))  # 6x6 matrix for 3D stress-strain relationship
    factor = E / (1 + nu) / (1 - 2 * nu)
    C[0, 0] = C[1, 1] = C[2, 2] = factor * (1 - nu)
    C[3, 3] = C[4, 4] = C[5, 5] = E / 2 / (1 + nu)
    C[0, 1] = C[1, 0] = C[0, 2] = C[2, 0] = C[1, 2] = C[2, 1] = factor * nu
    return C


def compute_element_stiffness(mesh, E, nu, nodes):
    """Compute Element Stiffness

    Args:
        mesh (pyvista.DataSet): The finite element mesh object that contains the geometry and connectivity information of the entire domain.
        E (float): Young's modulus
        nu (float): Poisson's ratio
        nodes (float): points on the mesh

    Returns:
        matrix: array of element stiffness
    """
    dN_dxi = cp.array([[-1, -1, -1], [1, 0, 0], [0, 1, 0], [0, 0, 1]])

    element_nodes = cp.array(mesh.points[nodes])  # Ensure this is CuPy
    J = compute_jacobian(element_nodes, dN_dxi)
    det_J = cp.linalg.det(J)

    if cp.abs(det_J) < 1e-12:
        print("Warning: Degenerate element detected. Skipping element.")
        return None

    J_inv = cp.linalg.inv(J)
    B = compute_B_matrix(J_inv, dN_dxi)
    C = compute_C_matrix(E, nu)
    K_elem = cp.dot(B.T, cp.dot(C, B)) * det_J
    return K_elem


def assemble_global_stiffness_efficient(K_global, K_elem, element):
    """Efficiently assembles the global stiffness matrix for a finite element model by adding the contributions from an element's stiffness matrix using sparse matrix operations.

    Args:
        K_global (cpx_sparse.csr_matrix): The global stiffness matrix in CSR (Compressed Sparse Row) format.
        K_elem (cpx_sparse.csr_matrix): The element stiffness matrix as a CuPy array of shape (num_nodes * num_dofs_per_node, num_nodes * num_dofs_per_node).
        element (cpx_sparse.csr_matrix):  An array or list of integers representing the global node indices that define the finite element. Each entry corresponds to a node in the global mesh.

    Returns:
        matrix: The updated global stiffness matrix after adding the contributions from the provided element's stiffness matrix. The input matrix is modified in place and returned for convenience.
    """
    num_dofs_per_node = 3
    num_nodes = len(element)

    # Compute global indices
    global_indices = cp.array(
        [int(node * num_dofs_per_node) for node in element], dtype=cp.int32
    )

    # Pre-allocate arrays for data, rows, and cols
    total_entries = (num_nodes * num_dofs_per_node) ** 2
    data = cp.zeros(total_entries, dtype=K_elem.dtype)
    rows = cp.zeros(total_entries, dtype=cp.int32)
    cols = cp.zeros(total_entries, dtype=cp.int32)

    # Use vectorized operations to compute indices
    node_indices = cp.arange(num_nodes)
    dof_indices = cp.arange(num_dofs_per_node)

    # Generate all combinations of node and DOF indices using meshgrid
    node_i, node_j = cp.meshgrid(node_indices, node_indices)
    dof_k, dof_l = cp.meshgrid(dof_indices, dof_indices)

    # Fill data array by flattening K_elem
    data[:] = K_elem.ravel()

    # Compute global indices for rows and cols
    rows[:] = global_indices[node_i.ravel()] + dof_k.ravel()
    cols[:] = global_indices[node_j.ravel()] + dof_l.ravel()

    # Create a COO matrix and add it to the global stiffness matrix
    K_global += cpx_sparse.coo_matrix(
        (data, (rows, cols)), shape=K_global.shape
    ).tocsr()

    return K_global


def compute_global_stiffness_matrix(mesh, E, nu):
    """Computes the global stiffness matrix for a finite element mesh using the material properties and element stiffness matrices. The global stiffness matrix is assembled by iterating over all elements in the mesh and summing their contributions.

    Args:
        mesh (pyvista.DataSet): The finite element mesh object that contains the geometry and connectivity information of the entire domain.
        E (float): Young's modulus
        nu (float): Poisson's ratio

    Returns:
        matrix: The global stiffness matrix in CSR (Compressed Sparse Row) format.
    """
    print("Computing global stiffness matrix...")
    K_global = cpx_sparse.coo_matrix(
        (mesh.n_points * 3, mesh.n_points * 3)
    )  # Start with COO format

    for i in tqdm(range(mesh.n_cells)):
        cell = mesh.get_cell(i)
        nodes = np.array(cell.point_ids).astype(int)  # Use NumPy array for indexing
        K_elem = compute_element_stiffness(mesh, E, nu, nodes)

        if K_elem is not None:
            K_global = assemble_global_stiffness_efficient(K_global, K_elem, nodes)

    print("Global stiffness matrix computed.")
    return K_global.tocsr()  # Convert to CSR format after assembly


# Apply Loads
def apply_gravity_load(mesh, density):
    """applies gravity loads

    Args:
        mesh (pyvista.DataSet): The finite element mesh object that contains the geometry and connectivity information of the entire domain.
        density (float): Density in kg/m³

    Returns:
        matrix: returns weight per unit volume matrix
    """
    print("Applying gravity load...")
    F_global = cp.zeros(mesh.n_points * 3)
    F_global[2::3] -= density * 9.81  # Apply gravity to the z-axis (index 2)
    print("Gravity load applied.")
    return F_global


def apply_pore_pressure(mesh, water_table_depth, pore_pressure):
    """Applies pore pressure to the global force vector in a finite element mesh based on a given water table depth.

    Args:
        mesh (pyvista.DataSet): The finite element mesh object that contains the geometry and connectivity information of the entire domain.
        water_table_depth (float): The depth of the water table relative to the global coordinate system.
        pore_pressure (float): The magnitude of the pore pressure to apply at nodes below the water table depth.

    Returns:
        array:  A 1D CuPy array representing the global force vector with pore pressure applied.
    """
    print("Applying pore pressure...")
    points = cp.array(mesh.points)  # Ensure points are a 2D CuPy array
    F_global = cp.zeros(mesh.n_points * 3)
    F_global[cp.where(points[:, 2] < water_table_depth)[0] * 3 + 2] += pore_pressure
    print("Pore pressure applied.")
    return F_global


def apply_surcharge_load(mesh, surcharge_load):
    """Applies a surcharge load to the global force vector in a finite element mesh.

    Args:
        mesh (pyvista.DataSet): The finite element mesh object that contains the geometry and connectivity information of the entire domain.
        surcharge_load (float):  The magnitude of the surcharge load to apply at nodes on the top surface of the mesh.

    Returns:
        array: A 1D CuPy array representing the global force vector with the surcharge load applied.
    """
    print("Applying surcharge load...")
    points = cp.array(mesh.points)  # Ensure points are a 2D CuPy array
    F_global = cp.zeros(mesh.n_points * 3)
    max_z = cp.max(points[:, 2])  # Ensure max_z is computed with CuPy
    F_global[cp.where(points[:, 2] == max_z)[0] * 3 + 2] += surcharge_load
    print("Surcharge load applied.")
    return F_global


def apply_seismic_load(mesh, seismic_coefficient):
    """Applies a seismic load to the global force vector in a finite element mesh.

    Args:
        mesh (pyvista.DataSet): The finite element mesh object that contains the geometry and connectivity information of the entire domain.
        seismic_coefficient (float): Seismic load factor

    Returns:
        array: A 1D CuPy array representing the global force vector with the seismic load applied.
    """
    print("Applying seismic load...")
    points = cp.array(mesh.points)  # Ensure points are a 2D CuPy array
    F_global = cp.zeros(mesh.n_points * 3)
    F_global[2::3] += seismic_coefficient * points[:, 2]
    print("Seismic load applied.")
    return F_global


def apply_loads(
    mesh, density, water_table_depth, pore_pressure, surcharge_load, seismic_coefficient
):
    """Applies various loads to the global force vector in a finite element mesh.

    Args:
        mesh (pyvista.DataSet): mesh (mesh object): The finite element mesh object that contains the geometry and connectivity information of the entire domain.
        density (float): Density in kg/m³
        water_table_depth (flaot): The depth of the water table relative to the global coordinate system.
        pore_pressure (float): The magnitude of the pore pressure to apply at nodes below the water table depth.
        surcharge_load (float): The magnitude of the surcharge load to apply at nodes on the top surface of the mesh.
        seismic_coefficient (flaot): Seismic load factor

    Returns:
        array: A 1D CuPy array representing the global force vector with all loads (gravity, pore pressure, surcharge and seismic) applied.
    """
    print("Applying all loads...")
    F_global = cp.zeros(mesh.n_points * 3)
    F_global += apply_gravity_load(mesh, density)
    F_global += apply_pore_pressure(mesh, water_table_depth, pore_pressure)
    F_global += apply_surcharge_load(mesh, surcharge_load)
    F_global += apply_seismic_load(mesh, seismic_coefficient)
    print("All loads applied.")
    return F_global

In [ ]:
# Identify Fixed Nodes
def identify_fixed_nodes(mesh):
    """Identifies fixed nodes in a finite element mesh based on specified boundary conditions.

    Args:
        mesh (pyvista.DataSet): The finite element mesh object that contains the geometry and connectivity information of the entire domain.

    Returns:
        array: An array of indices representing the fixed nodes in the mesh.
    """
    fixed_nodes = []

    # Find minimum and maximum coordinates
    min_z = mesh.points[:, 2].min()
    min_x = mesh.points[:, 0].min()
    max_x = mesh.points[:, 0].max()
    min_y = mesh.points[:, 1].min()
    max_y = mesh.points[:, 1].max()

    # Loop through each node and check if it's a fixed node
    for i, node in enumerate(mesh.points):
        # Fix base mesh nodes (minimum elevation)
        if node[2] == min_z:
            fixed_nodes.append(i)
        # Fix nodes on vertical sides of pit
        elif node[0] == min_x or node[0] == max_x:
            fixed_nodes.append(i)
        elif node[1] == min_y or node[1] == max_y:
            fixed_nodes.append(i)

    return fixed_nodes


# Apply Boundary Conditions
def apply_boundary_conditions(K_global, F_global, fixed_nodes):
    """Applies boundary conditions to the global stiffness matrix and global force vector for a finite element model.

    Args:
        K_global (matrix): The global stiffness matrix in CSR (Compressed Sparse Row) format or as a CuPy dense array.
        F_global (array): The global force vector as a CuPy array. It represents the applied forces at each node of the mesh.
        fixed_nodes (array): An array or list of integers representing the indices of nodes that are fixed in the mesh.

    Returns:
        tuple: K_global (cupyx.scipy.sparse.csr_matrix or cp.ndarray), F_global (cp.ndarray)
    """
    print("Applying boundary conditions...")

    num_dofs_per_node = 3
    fixed_dofs = cp.array(
        [
            node * num_dofs_per_node + i
            for node in fixed_nodes
            for i in range(num_dofs_per_node)
        ]
    )

    # Create a mask for non-fixed DOFs
    non_fixed_dofs = cp.ones(K_global.shape[0], dtype=bool)
    non_fixed_dofs[fixed_dofs] = False

    # Zero out the rows and columns for fixed DOFs in K_global
    K_global[fixed_dofs, :] = 0
    K_global[:, fixed_dofs] = 0

    # Set diagonal for fixed DOFs
    K_global[fixed_dofs, fixed_dofs] = 1

    # Zero out the corresponding entries in F_global
    F_global[fixed_dofs] = 0

    print(f"K_global shape: {K_global.shape}, F_global shape: {F_global.shape}")
    print("Boundary conditions applied.")
    return K_global, F_global


def matvec(K_global, x):
    """Performs matrix-vector multiplication.

    Args:
        K_global (cupyx.scipy.sparse.csr_matrix): The global stiffness matrix in CSR (Compressed Sparse Row) format or as a CuPy dense array.
        x (array): The vector to multiply with the global stiffness matrix, typically representing nodal displacements or forces.

    Returns:
        array: The result of the matrix-vector multiplication as a CuPy array.
    """
    return K_global.dot(x)


def solve_displacements(K_global, F_global, method="gmres"):
    """Solves for nodal displacements in a finite element mesh using iterative solvers.

    Args:
        K_global (cupyx.scipy.sparse.csr_matrix): _description_
        F_global (array): _description_
        method (str, optional): The iterative method to use for solving the linear system. Defaults to 'gmres'.

    Raises:
        ValueError: _description_

    Returns:
        array: The global displacement vector computed using the chosen iterative solver.
    """
    print(f"Solving for displacements using {method.upper()} on GPU...")

    # Ensure the global stiffness matrix is in CSR format for efficient matrix-vector operations
    if not isinstance(K_global, csr_matrix):
        K_global = K_global.tocsr()

    # Define a matrix-vector product function for the solver
    K_operator = LinearOperator(K_global.shape, matvec=lambda x: matvec(x, K_global))

    print(f"K_global size: {K_global.shape}, F_global size: {F_global.shape}")

    if method == "gmres":
        # Using GMRES solver on GPU
        U_global, info = gmres(K_operator, F_global, tol=1e-8, maxiter=3135)
    elif method == "bicgstab":
        print("Falling back to CPU-based BiCGSTAB solver...")
        # Transfer to CPU for SciPy's BiCGSTAB
        K_global_cpu = K_global.get()  # Transfer to CPU
        F_global_cpu = cp.asnumpy(F_global)  # Transfer to CPU
        U_global_cpu, info = bicgstab(
            K_global_cpu, F_global_cpu, tol=1e-8, maxiter=3135
        )
        U_global = cp.asarray(U_global_cpu)  # Transfer back to GPU
    else:
        raise ValueError(f"Unknown method: {method}")

    if info != 0:
        print(f"{method.upper()} solver did not converge. Info: {info}")
    else:
        print(f"Displacements solved using {method.upper()}.")

    return U_global


def compute_stresses(mesh, U_global, E, nu):
    """Computes the stress for each element in a finite element mesh using the global displacement vector.

    Args:
        mesh (pyvista.DataSet): The finite element mesh object that contains the geometry and connectivity information of the entire domain.
        U_global (array): The global displacement vector as a CuPy array.
        E (float): Young's modulus
        nu (float): Poisson's ratio

    Returns:
        array: A CuPy array containing the maximum stress for each element in the mesh.
    """
    print("Computing stresses...")

    stresses = cp.zeros(mesh.n_cells)
    dN_dxi = cp.array([[-1, -1, -1], [1, 0, 0], [0, 1, 0], [0, 0, 1]])

    # Convert mesh points to a CuPy array once to avoid repeated conversions
    mesh_points = cp.array(mesh.points)

    # Precompute range for DOFs
    dof_range = cp.arange(3)

    for i in tqdm(range(mesh.n_cells)):
        cell = mesh.get_cell(i)
        nodes = cp.array(cell.point_ids, dtype=cp.int32)
        element_nodes = mesh_points[nodes]

        J = compute_jacobian(element_nodes, dN_dxi)
        det_J = cp.linalg.det(J)

        if cp.abs(det_J) < 1e-12:
            print(f"Warning: Degenerate element detected at cell {i}. Skipping.")
            continue

        J_inv = cp.linalg.inv(J)
        B = compute_B_matrix(J_inv, dN_dxi)
        C = compute_C_matrix(E, nu)

        # Efficiently gather U_elem using advanced indexing
        U_elem = U_global[(nodes[:, None] * 3 + dof_range).ravel()]

        epsilon = cp.dot(B, U_elem)
        sigma = cp.dot(C, epsilon)
        stresses[i] = cp.max(sigma)

    print("Stresses computed.")
    return stresses


def compute_stress_tensor(mesh, U_global, E, nu):
    """Computes the stress tensor for each element in a finite element mesh using the global displacement vector.

    Args:
        mesh (pyvista.DataSet): The finite element mesh object that contains the geometry and connectivity information of the entire domain.
        U_global (array): The global displacement vector as a CuPy array.
        E (float): Young's modulus
        nu (float): Poisson's ratio

    Returns:
        array: A CuPy array containing the stress tensor components for each element in the mesh.
    """
    print("Computing stress tensor...")

    stresses = cp.zeros(
        (mesh.n_cells, 6)
    )  # Store 6 components of the stress tensor for each cell
    dN_dxi = cp.array([[-1, -1, -1], [1, 0, 0], [0, 1, 0], [0, 0, 1]])

    # Convert mesh points to a CuPy array once, to avoid repeated conversions
    mesh_points = cp.array(mesh.points)

    # Precompute range for DOFs
    dof_range = cp.arange(3)

    for i in tqdm(range(mesh.n_cells)):
        cell = mesh.get_cell(i)
        nodes = cp.array(cell.point_ids, dtype=cp.int32)
        element_nodes = mesh_points[nodes]

        J = compute_jacobian(element_nodes, dN_dxi)
        det_J = cp.linalg.det(J)

        if cp.abs(det_J) < 1e-12:
            print(f"Warning: Degenerate element detected at cell {i}. Skipping.")
            continue

        J_inv = cp.linalg.inv(J)
        B = compute_B_matrix(J_inv, dN_dxi)
        C = compute_C_matrix(E, nu)

        # Efficiently gather U_elem using advanced indexing
        U_elem = U_global[(nodes[:, None] * 3 + dof_range).ravel()]

        epsilon = cp.dot(B, U_elem)
        sigma = cp.dot(C, epsilon)

        # Store the stress tensor components
        stresses[i, 0] = sigma[0]  # σ_xx
        stresses[i, 1] = sigma[1]  # σ_yy
        stresses[i, 2] = sigma[2]  # σ_zz
        stresses[i, 3] = sigma[3]  # τ_xy
        stresses[i, 4] = sigma[4]  # τ_xz
        stresses[i, 5] = sigma[5]  # τ_yz

    print("Stress tensor computed.")
    return stresses


def calculate_normal_stress(stress_tensor, plane_normal):
    """Calculates the normal stress on a plane defined by a given normal vector.

    Args:
        stress_tensor (array): A 1D array containing the 6 components of the stress tensor:[σ_xx, σ_yy, σ_zz, τ_xy, τ_xz, τ_yz].
        plane_normal (array): A 1D array containing the 3 components of the normal vector defining the plane: [nx, ny, nz].

    Raises:
        ValueError: If the length of `stress_tensor` is not 6.

    Returns:
        float: The normal stress on the plane defined by `plane_normal`.
    """
    # Calculate the normal stress on a plane given by plane_normal
    nx, ny, nz = plane_normal
    if len(stress_tensor) != 6:
        raise ValueError(
            f"Expected 6 components in stress_tensor, got {len(stress_tensor)}"
        )

    sigma_xx, sigma_yy, sigma_zz = stress_tensor[:3]
    tau_xy, tau_xz, tau_yz = stress_tensor[3:]

    # Normal stress on the given plane
    normal_stress = (
        (nx**2) * sigma_xx
        + (ny**2) * sigma_yy
        + (nz**2) * sigma_zz
        + 2 * nx * ny * tau_xy
        + 2 * nx * nz * tau_xz
        + 2 * ny * nz * tau_yz
    )
    return normal_stress


def calculate_shear_strength(cohesion, normal_stress, friction_angle):
    """Calculates the shear strength of a material using the Mohr-Coulomb failure criterion.

    Args:
        cohesion (float or array): The cohesion of the material, which represents the shear strength at zero normal stress. Can be a scalar or a CuPy array.
        normal_stress (float or array): The normal stress acting on the plane of potential failure. Can be a scalar or a CuPy array.
        friction_angle (float or array): The internal friction angle of the material, in degrees. This parameter determines the slope of the failure envelope.

    Returns:
        float or array: The calculated shear strength using the Mohr-Coulomb failure criterion. The output will be a scalar if both `cohesion` and `normal_stress` are scalars, or a CuPy array if either input is a CuPy array.
    """
    # Convert friction angle to radians and compute tangent
    tan_phi = cp.tan(cp.radians(friction_angle))

    # Calculate shear strength using the Mohr-Coulomb criterion
    shear_strength = cohesion + normal_stress * tan_phi

    return shear_strength


def compute_fos(stresses, cohesions, friction_angles, mesh, plane_normal=(0, 0, 1)):
    """Calculates the Factor of Safety (FoS) for each element in a finite element mesh.

    Args:
        stresses (array): A CuPy array containing the stress tensors for each element in the mesh. Each stress tensor is represented by 6 components:[σ_xx, σ_yy, σ_zz, τ_xy, τ_xz, τ_yz].
        cohesions (array): A CuPy array containing the cohesion values for each element in the mesh.
        friction_angles (array): A CuPy array containing the internal friction angles (in degrees) for each element in the mesh.
        mesh (pyvista.DataSet): The finite element mesh object that contains the geometry and connectivity information of the entire domain.
        plane_normal (tuple, optional): A tuple representing the normal vector of the plane on which to compute the normal stress. Defaults to (0, 0, 1).

    Raises:
        ValueError: If the stress tensor for any cell does not have the correct dimensions or size.

    Returns:
        array: A CuPy array containing the Factor of Safety for each element in the mesh.
    """
    print("Calculating Factor of Safety (FoS)...")

    # Initialize FoS array with infinity to handle any potential divisions by zero
    fos = cp.full(mesh.n_cells, float("inf"))

    # Iterate over each cell to calculate FoS
    for i in tqdm(range(mesh.n_cells)):
        # Extract the full stress tensor for the current cell
        stress_tensor = stresses[i]

        # Ensure the stress_tensor has the correct dimensions
        if stress_tensor.ndim != 1 or stress_tensor.size != 6:
            raise ValueError(
                f"Stress tensor for cell {i} has incorrect dimensions or size: {stress_tensor.shape}"
            )

        # Calculate the normal stress on the specified plane (e.g., horizontal plane)
        normal_stress = calculate_normal_stress(stress_tensor, plane_normal)

        if normal_stress > 0:
            # Get the spatially varying material properties for the current cell
            cohesion = cohesions[i]
            friction_angle = friction_angles[i]

            # Calculate shear strength for the element
            shear_strength = calculate_shear_strength(
                cohesion, normal_stress, friction_angle
            )

            # Calculate FoS for the element
            fos[i] = shear_strength / normal_stress

    print("Factor of Safety (FoS) calculated.")
    return fos

In [ ]:
mesh = load_volume_mesh(filepath)
assign_material_properties(mesh, cohesion, friction_angle, density, E, nu)
K_global = compute_global_stiffness_matrix(mesh, E, nu)
F_global = apply_loads(
    mesh, density, water_table_depth, pore_pressure, surcharge_load, seismic_coefficient
)

In [ ]:
# Use the function to get fixed nodes
fixed_nodes = identify_fixed_nodes(mesh)

K_global, F_global = apply_boundary_conditions(K_global, F_global, fixed_nodes)

U_global = solve_displacements(K_global, F_global)

stresses = compute_stresses(mesh, U_global)
# Now call the updated compute_stress_tensor function
stresses = compute_stress_tensor(mesh, U_global, E, nu)

# Compute the stress tensor from your FEM analysis
stresses = compute_stress_tensor(mesh, U_global, E, nu)

# Define material properties (cohesion and friction angle)
cohesion = cp.array(
    [25e3] * mesh.n_cells
)  # Replace with actual cohesion values from your FEM model
friction_angles = cp.array(
    [35] * mesh.n_cells
)  # Replace with actual friction angles from your FEM model

# Calculate FoS for the entire mesh using the actual stresses from FEM
fos = compute_fos(stresses, cohesion, friction_angles, mesh)

# Advance Identification of Longest Connected Component Failure Surfaces

### 1. Thresholding FoS

In [ ]:
def plot_fixed_nodes(mesh, fixed_nodes, point_size=10, mesh_opacity=0.5):
    """Plots a finite element mesh and highlights the fixed nodes.

    Args:
        mesh (pyvista.DataSet): The finite element mesh object that contains the geometry and connectivity information of the entire domain.
        fixed_nodes (array): An array or list of indices representing the fixed nodes in the mesh.
        point_size (int, optional): The size of the points used to highlight the fixed nodes. Default is 10. Defaults to 10.
        mesh_opacity (float, optional): The opacity of the mesh. Value should be between 0 (completely transparent) and 1 (completely opaque). Defaults to 0.5.
    """
    # Initialize PyVista plotter with a specified window size
    plotter = pv.Plotter(window_size=(1000, 600))
    plotter.add_text("Fixed Nodes", font_size=12)

    # Plot the entire mesh with adjustable opacity
    plotter.add_mesh(mesh, color="white", show_edges=True, opacity=mesh_opacity)

    # Highlight the fixed nodes
    fixed_points = mesh.points[fixed_nodes]
    plotter.add_points(
        fixed_points, color="red", point_size=point_size, render_points_as_spheres=True
    )

    # Display the plot
    plotter.show()


# Call the function to plot the mesh with fixed nodes highlighted
plot_fixed_nodes(mesh, fixed_nodes)

In [ ]:
# Convert from CuPy to NumPy and handle infinite values
fos_values = cp.asnumpy(fos)
max_fos_value = 100000
min_fos_value = 0
# Replace infinite values with the maximum FoS value
fos_values = np.where(np.isinf(fos_values), max_fos_value, fos_values)

# Ensure FoS values are within the specified range
fos_values = np.clip(fos_values, min_fos_value, max_fos_value)

# Define the boundaries for the safety factors
bound1 = 1.0  # 5th value
bound2 = 1.3  # 10th value
bound3 = 1.5  # 15th value

# Calculate the number of intervals between each specified boundary
num_intervals1 = 5
num_intervals2 = 5
num_intervals3 = 5
num_intervals4 = 1

# Create the bounds array using equal quantiles for all intervals
bounds1 = np.quantile(fos_values, np.linspace(0, 0.2, num_intervals1 + 1))
bounds2 = np.quantile(fos_values, np.linspace(0.2, 0.4, num_intervals2 + 1))
bounds3 = np.quantile(fos_values, np.linspace(0.4, 0.6, num_intervals3 + 1))
bounds4 = np.quantile(fos_values, np.linspace(0.6, 1, num_intervals4 + 1))

# Insert bound1, bound2, and bound3 at their original positions
bounds = np.concatenate(
    [bounds1[1:], [bound1], bounds2[1:], [bound2], bounds3[1:], [bound3], bounds4[1:]]
)

# Ensure bounds are monotonically increasing
bounds = np.sort(bounds)

# Create the colormap and normalization
cmap = sns.color_palette("Spectral", as_cmap=True)
norm = BoundaryNorm(bounds, ncolors=cmap.N)

# Assign the FoS values to the mesh
mesh.cell_data["FoS"] = fos_values

# Use PyVista to plot the mesh with the assigned FoS values
plotter = pv.Plotter(window_size=(800, 600))
plotter.add_text("Factor of Safety (FoS)", font_size=12)

# Plot the mesh with the custom colormap and normalization
plotter.add_mesh(
    mesh,
    scalars="FoS",
    cmap=cmap,
    show_edges=True,
    clim=(np.min(bounds), np.max(bounds)),
)
plotter.add_scalar_bar("FoS", n_labels=len(bounds), color="black")

# Display the plot
plotter.show()

# Print the minimum and maximum FoS values for reference
print(f"Minimum FoS: {np.min(fos_values)}")
print(f"Maximum FoS: {np.max(fos_values)}")

In [ ]:
def identify_failing_elements(fos, threshold=1.0):
    """
    Identify elements with a Factor of Safety (FoS) below the specified threshold.

    Args:
        fos (cp.ndarray): Array of FoS values for each element.
        threshold (float): Threshold value for FoS to identify failing elements.

    Returns:
        cp.ndarray: Mask array indicating elements at risk of failure.
    """
    print(f"Identifying elements with FoS ≤ {threshold}...")

    # Directly use a Boolean mask to identify failing elements
    failing_elements = fos <= threshold

    # Use the sum of the Boolean array to count the number of failing elements
    num_failing_elements = cp.count_nonzero(failing_elements)

    print(f"Number of failing elements identified: {num_failing_elements}")
    return failing_elements

### 2. Stress and Strain Analysis

In [ ]:
def analyze_stress_strain(stress_tensor, strain_tensor, yield_stress, ultimate_stress):
    """
    Analyze stress and strain fields to identify areas with significant plastic deformation
    or where stress limits are exceeded.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        stress_tensor (cp.ndarray): Stress tensor for each element.
        strain_tensor (cp.ndarray): Strain tensor for each element.
        yield_stress (float): Yield stress of the material.
        ultimate_stress (float): Ultimate stress of the material.

    Returns:
        dict: Dictionaries containing masks for plastic deformation and high shear stress.
    """
    print("Analyzing stress and strain fields...")

    # Identify plastic deformation using vectorized operations
    plastic_deformation_mask = cp.any(strain_tensor > yield_stress, axis=1)

    # Calculate shear stress components using vectorized operations
    shear_stress = cp.sqrt(
        stress_tensor[:, 3] ** 2 + stress_tensor[:, 4] ** 2 + stress_tensor[:, 5] ** 2
    )

    # Identify high shear stress areas
    high_shear_stress_mask = shear_stress > ultimate_stress

    print("Stress and strain analysis completed.")

    return {
        "plastic_deformation": plastic_deformation_mask,
        "high_shear_stress": high_shear_stress_mask,
    }

In [ ]:
def analyze_stress_strain_optimized(
    stress_tensor, strain_tensor, yield_stress, ultimate_stress
):
    """
    Optimized analysis of stress and strain fields to identify areas with significant plastic deformation
    or where stress limits are exceeded.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        stress_tensor (cp.ndarray): Stress tensor for each element.
        strain_tensor (cp.ndarray): Strain tensor for each element.
        yield_stress (float): Yield stress of the material.
        ultimate_stress (float): Ultimate stress of the material.

    Returns:
        dict: Dictionaries containing masks for plastic deformation and high shear stress.
    """
    print("Analyzing stress and strain fields (Optimized)...")

    # Identify plastic deformation using vectorized operations
    plastic_deformation_mask = cp.any(strain_tensor > yield_stress, axis=1)

    # Calculate shear stress components and identify high shear stress
    shear_stress = cp.sqrt(
        stress_tensor[:, 3] ** 2 + stress_tensor[:, 4] ** 2 + stress_tensor[:, 5] ** 2
    )
    high_shear_stress_mask = shear_stress > ultimate_stress

    print("Optimized stress and strain analysis completed.")

    return {
        "plastic_deformation": plastic_deformation_mask,
        "high_shear_stress": high_shear_stress_mask,
    }

In [ ]:
def calculate_mesh_shape(mesh):
    """
    Calculate the shape of the mesh grid directly from a structured or semi-structured 3D mesh.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.

    Returns:
        tuple: The estimated shape of the mesh (nx, ny, nz).
    """
    # Get the bounds of the mesh
    bounds = mesh.bounds  # (xmin, xmax, ymin, ymax, zmin, zmax)

    # Find unique coordinate values along each axis
    x_coords = np.unique(mesh.points[:, 0])
    y_coords = np.unique(mesh.points[:, 1])
    z_coords = np.unique(mesh.points[:, 2])

    # Calculate the number of unique points along each axis to determine grid dimensions
    nx = len(x_coords)
    ny = len(y_coords)
    nz = len(z_coords)

    print(f"Calculated mesh shape: (nx, ny, nz) = ({nx}, {ny}, {nz})")

    return (nx, ny, nz)

### 3. Connected Component Analysis

In [ ]:
def connected_component_analysis(mesh, failing_elements):
    """
    Perform connected component analysis to detect coherent failure surfaces.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failing_elements (cp.ndarray): Indices of elements with FoS ≤ 1.0.

    Returns:
        np.ndarray: Array indicating the connected component label for each failing element.
    """
    print("Performing connected component analysis...")

    # Convert failing_elements from CuPy to NumPy array
    failing_elements_np = failing_elements.get()

    # Initialize a mask array to identify failing elements
    element_mask = np.zeros(mesh.n_cells, dtype=bool)
    element_mask[failing_elements_np] = True

    # Define 3D connectivity structure for 3D connected component analysis
    structure = np.ones((3, 3, 3), dtype=int)

    # Perform connected component analysis
    labeled_array, num_features = label(element_mask, structure=structure)

    print(f"Number of connected components identified: {num_features}")
    return labeled_array

In [ ]:
failing_elements = identify_failing_elements(fos, threshold=10)
connected_components = connected_component_analysis(mesh, failing_elements)
# Convert the connected components result from CuPy to NumPy
connected_components_np = (
    connected_components  # Already a NumPy array from the function
)

# Create a colormap for visualization of different connected components
unique_labels = np.unique(connected_components_np)
n_labels = len(unique_labels)

# Add the connected components data as cell data to the mesh
mesh.cell_data["Connected Components"] = connected_components_np

# Initialize PyVista plotter
plotter = pv.Plotter(window_size=(800, 600))
plotter.add_text("Connected Component Analysis of Failing Elements", font_size=12)

# Plot the mesh with different colors for each connected component
plotter.add_mesh(
    mesh,
    scalars="Connected Components",
    show_edges=True,
    cmap="tab10",
    clim=[0, n_labels - 1],
    scalar_bar_args={"title": "Connected Components"},
)

# Show the plot
plotter.show()

### 4. Surface Extraction and Identification of Failure Surfaces

In [ ]:
def extract_failure_surfaces(mesh, connected_components):
    """
    Extract failure surfaces based on connected components.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        connected_components (np.ndarray): Array of connected component labels.

    Returns:
        list: A list of extracted surfaces for each connected component.
    """
    print("Extracting failure surfaces...")

    failure_surfaces = []

    # Ensure connected_components is a NumPy array
    connected_components = np.asarray(connected_components)

    # Iterate over each unique label in the connected components, skipping label 0
    unique_labels = np.unique(connected_components)
    for component_label in unique_labels:
        if component_label == 0:
            continue  # Skip non-failing elements

        # Extract the cells for the current connected component
        mask = connected_components == component_label
        surface = mesh.extract_cells(mask)
        failure_surfaces.append(surface)

    print(f"Number of failure surfaces extracted: {len(failure_surfaces)}")
    return failure_surfaces

    print(f"Number of failure surfaces extracted: {len(failure_surfaces)}")
    return failure_surfaces


failure_surfaces = extract_failure_surfaces(mesh, connected_components)

In [ ]:
# Generate a list of distinct colors using matplotlib's colormap
colors = cm.get_cmap("tab20", len(failure_surfaces))

# Initialize PyVista plotter
plotter = pv.Plotter(window_size=(800, 600))
plotter.add_text("Extracted Failure Surfaces", font_size=12)

# Iterate over each extracted failure surface and plot
for i, surface in enumerate(failure_surfaces):
    # Use the generated colors
    color = colors(i)[:3]  # matplotlib returns RGBA, we need RGB
    plotter.add_mesh(surface, color=color, show_edges=True)

# Show the plot
plotter.show()

### 5. Visualization and Interpretation

In [ ]:
def visualize_failure_analysis(
    mesh, failure_surfaces, stress_tensor, strain_tensor, fos
):
    """
    Visualize failure surfaces and stress/strain distributions.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failure_surfaces (list): List of extracted failure surfaces.
        stress_tensor (cp.ndarray): Stress tensor for each element.
        strain_tensor (cp.ndarray): Strain tensor for each element.
        fos (cp.ndarray): Factor of Safety for each element.
    """
    print("Visualizing failure analysis results...")

    plotter = pv.Plotter()

    # Visualize the original mesh with low opacity for background context
    plotter.add_mesh(
        mesh, opacity=0.1, show_edges=True, color="lightgrey", label="Original Mesh"
    )

    # Visualize each failure surface
    for i, surface in enumerate(failure_surfaces):
        plotter.add_mesh(surface, color="red", label=f"Failure Surface {i+1}")

    # Convert CuPy arrays to NumPy for visualization in PyVista
    stress_tensor_np = cp.asnumpy(stress_tensor)
    strain_tensor_np = cp.asnumpy(strain_tensor)
    fos_np = cp.asnumpy(fos)

    # Add scalar bars and overlay stress, strain, and FoS on the mesh
    plotter.add_mesh(
        mesh,
        scalars=stress_tensor_np[:, 0],
        cmap="coolwarm",
        opacity=0.6,
        label="Stress Distribution",
    )
    plotter.add_scalar_bar("Stress", title="Stress (MPa)")

    plotter.add_mesh(
        mesh,
        scalars=strain_tensor_np[:, 0],
        cmap="viridis",
        opacity=0.6,
        label="Strain Distribution",
    )
    plotter.add_scalar_bar("Strain", title="Strain (%)")

    plotter.add_mesh(
        mesh, scalars=fos_np, cmap="jet", opacity=0.6, label="FoS Distribution"
    )
    plotter.add_scalar_bar("Factor of Safety", title="FoS")

    plotter.show()

### 6. Slope Stability and Failure Mode Analysis

In [ ]:
def slope_stability_analysis(
    mesh, failure_surfaces, stress_tensor, strain_tensor, fos, external_factors
):
    """
    Perform a detailed slope stability and failure mode analysis based on the provided mesh data, failure surfaces, and external factors.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model, typically representing a slope in geotechnical analysis.
        failure_surfaces (list): A list of extracted failure surfaces, each potentially representing a different failure mechanism.
        stress_tensor (cp.ndarray): A tensor containing stress values for each element in the mesh.
        strain_tensor (cp.ndarray): A tensor containing strain values for each element in the mesh.
        fos (cp.ndarray): An array containing the calculated Factor of Safety for each element.
        external_factors (dict): A dictionary containing external factors such as seismic coefficients, water pressures, etc., that affect slope stability.

    Returns:
        dict: A dictionary containing results of the slope stability analysis, including identified failure modes and critical surfaces.
    """
    print("Performing slope stability analysis...")

    results = {"failure_modes": [], "critical_surfaces": []}

    # Evaluate each failure surface for potential failure modes and criticality
    for index, surface in enumerate(failure_surfaces):
        # Assume the index corresponds to mesh cell indices for simplicity in this example
        surface_stress = stress_tensor[surface]
        surface_fos = fos[surface]

        # Identify if any part of the surface is below a FoS threshold
        critical = np.any(surface_fos < 1.0)
        if critical:
            results["critical_surfaces"].append(surface)

        # Identify failure modes based on stress characteristics
        shear_stresses = cp.sqrt(
            cp.sum(surface_stress[:, 3:] ** 2, axis=1)
        )  # Simplified shear stress calculation
        predominant_failure_mode = (
            "sliding"
            if cp.any(
                shear_stresses > external_factors.get("shear_stress_threshold", 100)
            )
            else "settling"
        )

        results["failure_modes"].append(
            {
                "surface_index": index,
                "mode": predominant_failure_mode,
                "is_critical": critical,
            }
        )

    print("Slope stability analysis completed.")
    return results

## Visualization Code for Results

### 1. Visualizing the Mesh and Failure Surfaces

In [ ]:
def visualize_mesh_and_failure_surfaces(
    mesh, failure_surfaces, mesh_color="white", mesh_opacity=0.5, surface_color="red"
):
    """
    Visualize the original mesh and the extracted failure surfaces with adjustable visual properties.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failure_surfaces (list): List of extracted failure surfaces.
        mesh_color (str): Color of the mesh. Default is "white".
        mesh_opacity (float): Opacity of the mesh, ranging from 0 (transparent) to 1 (opaque). Default is 0.5.
        surface_color (str): Color used for failure surfaces. Default is "red".
    """
    print("Visualizing mesh and failure surfaces...")

    plotter = pv.Plotter()

    # Visualize the original mesh with adjustable opacity and color
    plotter.add_mesh(
        mesh,
        color=mesh_color,
        opacity=mesh_opacity,
        show_edges=True,
        label="Original Mesh",
    )

    # Visualize each failure surface with a distinct color and customizable options
    for i, surface in enumerate(failure_surfaces):
        plotter.add_mesh(
            surface,
            color=surface_color,
            show_edges=True,
            label=f"Failure Surface {i+1}",
        )

    plotter.add_legend()
    plotter.show()

### 2. Visualizing Stress and Strain Fields

In [ ]:
import pyvista as pv
import cupy as cp
import numpy as np


def visualize_stress_strain_fields(mesh, stress_tensor, strain_tensor):
    """
    Visualize stress and strain fields on the mesh, enhanced with performance optimizations and error handling.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        stress_tensor (cp.ndarray): Stress tensor for each element. Shape should be (n_elements, n_stress_components).
        strain_tensor (cp.ndarray): Strain tensor for each element. Shape should be (n_elements, n_strain_components).
    """
    print("Visualizing stress and strain fields...")

    # Validate inputs
    if not isinstance(mesh, pv.UnstructuredGrid):
        raise ValueError("The mesh must be a PyVista UnstructuredGrid.")
    if stress_tensor.shape[1] != 3 or strain_tensor.shape[1] != 3:
        raise ValueError(
            "Stress and strain tensors must have 3 components per element."
        )

    plotter = pv.Plotter(shape=(1, 2))  # Two plots side by side

    # Efficiently convert tensors from CuPy to NumPy
    stress_magnitude = cp.linalg.norm(stress_tensor, axis=1).get()
    strain_magnitude = cp.linalg.norm(strain_tensor, axis=1).get()

    # Stress field visualization
    plotter.subplot(0, 0)
    plotter.add_text("Stress Field", font_size=12)
    plotter.add_mesh(mesh, scalars=stress_magnitude, cmap="coolwarm", show_edges=True)
    plotter.add_scalar_bar("Stress Magnitude", format="%.2e")

    # Strain field visualization
    plotter.subplot(0, 1)
    plotter.add_text("Strain Field", font_size=12)
    plotter.add_mesh(mesh, scalars=strain_magnitude, cmap="viridis", show_edges=True)
    plotter.add_scalar_bar("Strain Magnitude", format="%.2e")

    plotter.show()


# Example usage
visualize_stress_strain_fields(mesh, stresses, stresses)

### 3. Visualizing Factor of Safety (FoS) Distribution

In [ ]:
def visualize_fos_distribution(mesh, fos, custom_range=None):
    """
    Visualize the Factor of Safety (FoS) distribution on the mesh with options for customizing the color map range.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        fos (cp.ndarray): Factor of Safety for each element.
        custom_range (tuple, optional): Min and max values to define the color map scale. Default is None.
    """
    print("Visualizing FoS distribution...")

    # Convert CuPy array to NumPy for PyVista compatibility
    fos_np = cp.asnumpy(fos)

    plotter = pv.Plotter()

    # Set up the scalar range based on FoS values or use custom range if provided
    scalar_range = custom_range if custom_range else (fos_np.min(), fos_np.max())

    # Set up a colormap that highlights critical values more distinctly
    cmap = "coolwarm"  # This colormap varies from blue (low) to red (high), good for safety factors

    plotter.add_mesh(
        mesh,
        scalars=fos_np,
        cmap=cmap,
        scalar_bar_args={"title": "Factor of Safety"},
        clim=scalar_range,
        show_edges=True,
    )

    # Enable user interaction for more detailed examination
    plotter.enable_zoom()
    plotter.enable_picking()

    plotter.show()


# Example usage
visualize_fos_distribution(mesh, fos)

### 4. Visualizing Connected Components of Failing Elements

In [ ]:
def visualize_connected_components(mesh, connected_components):
    """
    Enhanced visualization of connected components of failing elements on the mesh.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        connected_components (np.ndarray): Array of connected component labels.
    """
    print("Visualizing connected components of failing elements...")

    plotter = pv.Plotter()
    unique_labels = np.unique(connected_components)

    # Generate a list of colors for each component
    colors = pv.plotting.get_cmap(
        "viridis", len(unique_labels) - 1
    )  # Exclude the zero label

    # Add each connected component with a unique color
    color_index = 0
    for label in unique_labels:
        if label == 0:
            continue  # Skip non-failing elements

        component = mesh.extract_cells(connected_components == label)
        plotter.add_mesh(
            component,
            color=colors[color_index],
            show_edges=True,
            label=f"Component {label}",
        )
        color_index += 1

    plotter.add_legend()
    plotter.enable_zoom()  # Allow users to zoom into specific components
    plotter.enable_picking()  # Allow picking of components to display more details
    plotter.show()


# Example usage
visualize_connected_components(mesh, connected_components)

## Analyzing Geometry and Orientation of Failure Surfaces

### 1. Analyzing Geometry and Orientation of Failure Surfaces

In [ ]:
def analyze_failure_modes(failure_surfaces):
    """
    Enhanced analysis of the geometry and orientation of failure surfaces to determine likely failure modes.

    Args:
        failure_surfaces (list): List of extracted failure surfaces.

    Returns:
        list: A list of likely failure modes for each failure surface with detailed analysis.
    """
    print("Analyzing geometry and orientation of failure surfaces...")
    failure_modes = []

    for surface in failure_surfaces:
        try:
            # Ensure the surface is a PyVista UnstructuredGrid
            if not isinstance(surface, pv.UnstructuredGrid):
                raise TypeError(
                    "Each failure surface must be a PyVista UnstructuredGrid."
                )

            # Convert UnstructuredGrid to PolyData for normal computation
            surface_polydata = surface.extract_surface()

            # Compute normals for the surface if not already available
            surface_with_normals = surface_polydata.compute_normals(
                point_normals=True, cell_normals=False, inplace=False
            )

            # Access the computed point normals
            normals = surface_with_normals.point_data["Normals"]
            average_normal = np.mean(normals, axis=0)
            average_normal /= np.linalg.norm(average_normal)  # Normalize

            # Determine failure mode based on surface orientation
            if np.abs(average_normal[2]) > 0.9:  # Mostly vertical
                failure_mode = "Toppling"
            elif (
                np.abs(average_normal[0]) > 0.9 or np.abs(average_normal[1]) > 0.9
            ):  # Mostly horizontal
                failure_mode = "Sliding"
            else:
                failure_mode = "Complex Failure"

            failure_modes.append(failure_mode)
            print(f"Surface analyzed: Mode = {failure_mode}")

        except Exception as e:
            print(f"Error processing surface: {e}")
            failure_modes.append("Undetermined")

    return failure_modes


# Example usage
failure_modes = analyze_failure_modes(failure_surfaces)

In [ ]:
def analyze_failure_modes(failure_surfaces):
    """
    Analyze the geometry and orientation of failure surfaces to determine likely failure modes.

    Args:
        failure_surfaces (list): List of extracted failure surfaces.

    Returns:
        list: A list of likely failure modes for each failure surface.
    """
    print("Analyzing geometry and orientation of failure surfaces...")

    failure_modes = []

    for surface in failure_surfaces:
        # Convert UnstructuredGrid to PolyData for normal computation
        surface_polydata = surface.extract_surface()

        # Compute normals for the surface
        surface_with_normals = surface_polydata.compute_normals(
            point_normals=True, cell_normals=False, inplace=False
        )

        # Access the computed point normals
        normals = surface_with_normals.point_data["Normals"]
        average_normal = np.mean(normals, axis=0)
        average_normal = average_normal / np.linalg.norm(average_normal)  # Normalize

        # Print the average normal for inspection
        print(f"Average normal vector: {average_normal}")

        # Determine failure mode based on surface orientation
        if np.abs(average_normal[2]) > 0.9:  # Mostly vertical
            failure_mode = "Toppling"
        elif (
            np.abs(average_normal[0]) > 0.9 or np.abs(average_normal[1]) > 0.9
        ):  # Mostly horizontal
            failure_mode = "Sliding"
        else:
            failure_mode = "Slope Failure"

        failure_modes.append(failure_mode)
        print(f"Surface analyzed: Mode = {failure_mode}")

    return failure_modes


# Example usage
failure_modes = analyze_failure_modes(failure_surfaces)

### 2. Identifying Potential Shear Bands

In [ ]:
def identify_shear_bands(mesh, strain_tensor, shear_strain_threshold):
    """
    Enhanced identification of potential shear bands by analyzing areas with concentrated shear strain directly on the GPU.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        strain_tensor (cp.ndarray): Strain tensor for each element, assumed to be on the GPU.
        shear_strain_threshold (float): Threshold for detecting high shear strain.

    Returns:
        pv.UnstructuredGrid: A subset of the mesh representing potential shear bands.
    """
    print("Identifying potential shear bands...")

    # Ensure strain tensor is a CuPy array for GPU acceleration
    if not isinstance(strain_tensor, cp.ndarray):
        raise ValueError("strain_tensor must be a CuPy ndarray.")

    # Calculate the magnitude of shear strain using GPU
    shear_strain_magnitude = cp.sqrt(
        strain_tensor[:, 3] ** 2 + strain_tensor[:, 4] ** 2 + strain_tensor[:, 5] ** 2
    )

    # Identify elements with shear strain above the threshold
    shear_band_elements = cp.nonzero(shear_strain_magnitude > shear_strain_threshold)[
        0
    ].get()

    # Extract the shear band regions from the mesh
    shear_bands = mesh.extract_cells(shear_band_elements)

    print(f"Number of elements identified as shear bands: {len(shear_band_elements)}")
    return shear_bands


# Example usage
shear_bands = identify_shear_bands(mesh, stresses, shear_strain_threshold=0.05)

### 3. Slip Surface Analysis

In [ ]:
def slip_surface_analysis(mesh, U_global, displacement_threshold):
    """
    Detect potential slip surfaces by analyzing zones of continuous displacement.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        U_global (cp.ndarray): Displacement vector for each node in the mesh.
        displacement_threshold (float): Threshold for detecting significant displacements.

    Returns:
        pv.UnstructuredGrid: A subset of the mesh representing potential slip surfaces.
    """
    print("Performing slip surface analysis...")

    # Calculate the displacement magnitude for each node
    displacement_magnitude = cp.linalg.norm(U_global.reshape(-1, 3), axis=1)
    displacement_magnitude_np = cp.asnumpy(displacement_magnitude)

    # Identify elements where displacement exceeds the threshold
    slip_surface_elements = np.where(
        displacement_magnitude_np > displacement_threshold
    )[0]

    # Extract the slip surface regions from the mesh
    slip_surfaces = mesh.extract_cells(slip_surface_elements)

    print(
        f"Number of elements identified as slip surfaces: {len(slip_surface_elements)}"
    )
    return slip_surfaces


# Example usage
slip_surfaces = slip_surface_analysis(mesh, U_global, displacement_threshold=0.0001)

### 4. Visualization of Shear Bands and Slip Surfaces

In [ ]:
def visualize_shear_bands_and_slip_surfaces(
    mesh,
    shear_bands,
    slip_surfaces,
    mesh_color="white",
    mesh_opacity=0.1,
    shear_band_color="blue",
    slip_surface_color="orange",
):
    """
    Enhanced visualization of shear bands and slip surfaces on the mesh with customizable color and opacity settings.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        shear_bands (pv.UnstructuredGrid): Mesh subset representing shear bands.
        slip_surfaces (pv.UnstructuredGrid): Mesh subset representing slip surfaces.
        mesh_color (str): Color for the original mesh.
        mesh_opacity (float): Opacity for the original mesh.
        shear_band_color (str): Color for the shear bands.
        slip_surface_color (str): Color for the slip surfaces.
    """
    print("Visualizing shear bands and slip surfaces...")

    plotter = pv.Plotter()

    # Visualize the original mesh with customizable transparency and color
    plotter.add_mesh(
        mesh,
        color=mesh_color,
        opacity=mesh_opacity,
        show_edges=True,
        label="Original Mesh",
    )

    # Visualize shear bands with a specified color
    plotter.add_mesh(
        shear_bands, color=shear_band_color, show_edges=True, label="Shear Bands"
    )

    # Visualize slip surfaces with a specified color
    plotter.add_mesh(
        slip_surfaces, color=slip_surface_color, show_edges=True, label="Slip Surfaces"
    )

    plotter.add_legend()
    plotter.enable_zoom()
    plotter.enable_rotation()
    plotter.show()


# Example usage
visualize_shear_bands_and_slip_surfaces(mesh, shear_bands, slip_surfaces)

In [ ]:
# Assuming U_global is a CuPy array containing the global displacement vector already computed
# Convert U_global from CuPy to NumPy for easier handling with NumPy operations
U_global_np = U_global.get()


def get_displacement_vectors(mesh, failure_surfaces, U_global_np):
    """
    Calculate average displacement vectors for cells in each failure surface of the mesh.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model, not directly used but relevant if expansion needed.
        failure_surfaces (list of pv.UnstructuredGrid): List of mesh subsets, each representing a failure surface.
        U_global_np (numpy.ndarray): Flattened array of displacements for all nodes in the mesh. Expected to have a length that is a multiple of 3, as each node's displacement is represented by three consecutive elements (x, y, z displacements).

    Returns:
        list of numpy.ndarray: Each element of the list is a numpy array where each row represents the average displacement vector of all nodes within a cell of the corresponding failure surface.

    Raises:
        ValueError: If any node index in the surfaces is invalid or if `U_global_np` is not structured as expected.
    """

    if U_global_np.ndim != 1 or (len(U_global_np) % 3) != 0:
        raise ValueError(
            "Global displacement array must be a flat array with length a multiple of 3."
        )

    displacement_vectors = []
    for surface in failure_surfaces:
        # Preallocate array to store average displacements for each cell
        surface_vectors = np.zeros((surface.n_cells, 3))

        for i in range(surface.n_cells):
            cell = surface.get_cell(i)
            node_indices = np.array(cell.point_ids).astype(int)

            if np.any(node_indices >= len(U_global_np) // 3):
                raise ValueError(
                    f"Invalid node index found in cell {i}; indices exceed displacement array size."
                )

            # Efficiently fetch displacements for all nodes in the cell
            node_displacements = U_global_np[
                node_indices * 3 : (node_indices * 3 + 3)
            ].reshape(-1, 3)
            surface_vectors[i] = np.mean(node_displacements, axis=0)

        displacement_vectors.append(surface_vectors)

    return displacement_vectors


# Example usage
displacement_vectors = get_displacement_vectors(mesh, failure_surfaces, U_global_np)

In [ ]:
import numpy as np
import pyvista as pv
import cupy as cp


def identify_failing_elements(fos, threshold=1.0):
    """
    Identify elements with a Factor of Safety (FoS) below the specified threshold.

    Args:
        fos (cp.ndarray): Array of FoS values for each element.
        threshold (float): Threshold value for FoS to identify failing elements.

    Returns:
        cp.ndarray: Mask array indicating elements at risk of failure.
    """
    print(f"Identifying elements with FoS ≤ {threshold}...")
    failing_elements = cp.where(fos <= threshold)[0]
    print(f"Number of failing elements identified: {len(failing_elements)}")
    return failing_elements


def analyze_stress_strain_optimized(
    stress_tensor, strain_tensor, yield_stress, ultimate_stress
):
    """
    Optimized analysis of stress and strain fields to identify areas with significant plastic deformation
    or where stress limits are exceeded.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        stress_tensor (cp.ndarray): Stress tensor for each element.
        strain_tensor (cp.ndarray): Strain tensor for each element.
        yield_stress (float): Yield stress of the material.
        ultimate_stress (float): Ultimate stress of the material.

    Returns:
        dict: Dictionaries containing masks for plastic deformation and high shear stress.
    """
    print("Analyzing stress and strain fields (Optimized)...")

    # Identify plastic deformation using vectorized operations
    plastic_deformation_mask = cp.any(strain_tensor > yield_stress, axis=1)

    # Calculate shear stress components and identify high shear stress
    shear_stress = cp.sqrt(
        stress_tensor[:, 3] ** 2 + stress_tensor[:, 4] ** 2 + stress_tensor[:, 5] ** 2
    )
    high_shear_stress_mask = shear_stress > ultimate_stress

    print("Optimized stress and strain analysis completed.")

    return {
        "plastic_deformation": plastic_deformation_mask,
        "high_shear_stress": high_shear_stress_mask,
    }


def connected_component_analysis(mesh, failing_elements):
    """
    Perform connected component analysis to detect coherent failure surfaces.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failing_elements (cp.ndarray): Indices of elements with FoS ≤ 1.0.

    Returns:
        np.ndarray: Array indicating the connected component label for each failing element.
    """
    print("Performing connected component analysis...")

    # Convert failing_elements to a NumPy array explicitly
    failing_elements_np = failing_elements.get()

    # Initialize a mask array to identify failing elements
    element_mask = np.zeros(mesh.n_cells, dtype=bool)
    element_mask[failing_elements_np] = True

    # Perform connected component analysis
    structure = np.array([1, 1, 1])  # Define 1D connectivity for a 1D input
    labeled_array, num_features = label(element_mask, structure)

    print(f"Number of connected components identified: {num_features}")
    return labeled_array


def extract_failure_surfaces(mesh, connected_components):
    """
    Extract failure surfaces based on connected components.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        connected_components (np.ndarray): Array of connected component labels.

    Returns:
        list: A list of extracted surfaces for each connected component.
    """
    print("Extracting failure surfaces...")

    failure_surfaces = []

    # Ensure connected_components is a NumPy array
    if not isinstance(connected_components, np.ndarray):
        connected_components = np.array(connected_components)

    # Iterate over each unique label in the connected components
    for component_label in np.unique(connected_components):
        if component_label == 0:
            continue  # Skip non-failing elements

        # Extract the cells for the current connected component
        mask = connected_components == component_label
        cells = np.where(mask)[0]  # Get cell indices
        failure_surfaces.append(cells)

    print(f"Number of failure surfaces extracted: {len(failure_surfaces)}")
    return failure_surfaces


def analyze_cause_of_failure(failure_surfaces, stresses, strains, fos):
    """
    Analyze the causes of failure for each surface based on stress, strain, and FoS data.

    Args:
        failure_surfaces (list): List of extracted failure surfaces.
        stresses (cp.ndarray): Stress tensor for each element.
        strains (cp.ndarray): Strain tensor for each element.
        fos (cp.ndarray): Factor of Safety for each element.

    Returns:
        dict: Results of the cause of failure analysis.
    """
    print("Analyzing causes of failure...")

    failure_causes = []

    for surface in failure_surfaces:
        # Extract stress, strain, and FoS values for the current surface
        stress_values = stresses[surface]
        strain_values = strains[surface]
        fos_values = fos[surface]

        # Analyze stress and strain values
        stress_magnitude = cp.linalg.norm(stress_values, axis=1)
        strain_magnitude = cp.linalg.norm(strain_values, axis=1)

        # Determine cause of failure based on thresholds
        max_stress = cp.max(stress_magnitude)
        max_strain = cp.max(strain_magnitude)
        max_fos = cp.min(fos_values)

        cause = {"max_stress": max_stress, "max_strain": max_strain, "min_fos": max_fos}
        failure_causes.append(cause)

    print("Cause of failure analysis completed.")
    return failure_causes


def extract_failure_directions(failure_surfaces, mesh):
    """
    Extract failure directions based on the orientation of the failure surfaces.

    Args:
        failure_surfaces (list): List of extracted failure surfaces.
        mesh (pv.UnstructuredGrid): The mesh of the model.

    Returns:
        list: A list of failure directions for each failure surface.
    """
    print("Extracting failure directions...")

    failure_directions = []

    for surface_cells in failure_surfaces:
        # Extract the cells for the current surface
        surface = mesh.extract_cells(surface_cells)
        surface_polydata = surface.extract_surface()

        # Compute normals for the surface if not already available
        surface_with_normals = surface_polydata.compute_normals(
            point_normals=True, cell_normals=False, inplace=False
        )

        # Access the computed point normals
        normals = surface_with_normals.point_data["Normals"]
        average_normal = np.mean(normals, axis=0)
        average_normal = average_normal / np.linalg.norm(average_normal)  # Normalize

        # Determine failure direction based on surface orientation
        if np.abs(average_normal[2]) > 0.9:  # Mostly vertical
            direction = "Vertical"
        elif (
            np.abs(average_normal[0]) > 0.9 or np.abs(average_normal[1]) > 0.9
        ):  # Mostly horizontal
            direction = "Horizontal"
        else:
            direction = "Oblique"

        failure_directions.append(direction)
        print(f"Surface analyzed: Direction = {direction}")

    return failure_directions


def calculate_failure_magnitude(stress_tensor, strain_tensor, fos):
    """
    Calculate the magnitude of failure based on stress, strain, and FoS.

    Args:
        stress_tensor (cp.ndarray): Stress tensor for each element.
        strain_tensor (cp.ndarray): Strain tensor for each element.
        fos (cp.ndarray): Factor of Safety for each element.

    Returns:
        cp.ndarray: Magnitude of failure for each element.
    """
    print("Calculating failure magnitude...")

    # Calculate stress and strain magnitudes
    stress_magnitude = cp.linalg.norm(stress_tensor, axis=1)
    strain_magnitude = cp.linalg.norm(strain_tensor, axis=1)

    # Ensure no division by zero or invalid operations
    fos = cp.where(
        fos == 0, cp.nan, fos
    )  # Replace zero FoS with NaN to avoid division errors

    # Define failure magnitude as a combination of stress, strain, and FoS
    failure_magnitude = stress_magnitude + strain_magnitude - fos

    # Handle any NaN values that might arise
    failure_magnitude = cp.nan_to_num(
        failure_magnitude, nan=0.0, posinf=0.0, neginf=0.0
    )

    return failure_magnitude


# Example usage
failing_elements = identify_failing_elements(fos, threshold=1.0)
analysis_results_optimized = analyze_stress_strain_optimized(
    stresses, stresses, yield_stress=300e6, ultimate_stress=500e6
)
connected_components = connected_component_analysis(mesh, failing_elements)
failure_surfaces = extract_failure_surfaces(mesh, connected_components)
failure_causes = analyze_cause_of_failure(failure_surfaces, stresses, stresses, fos)
failure_directions = extract_failure_directions(failure_surfaces, mesh)
failure_magnitude = calculate_failure_magnitude(stresses, stresses, fos)

print("Failure causes:", failure_causes)
print("Failure directions:", failure_directions)
print("Failure magnitudes:", failure_magnitude)

In [ ]:
# Create a plotter object for 3D visualization
plotter = pv.Plotter()

# Add mesh to the plotter (if the mesh is large, consider subsampling)
plotter.add_mesh(mesh, color="white", opacity=1)

# Plot failure surfaces
for i, surface_cells in enumerate(failure_surfaces):
    surface = mesh.extract_cells(surface_cells)
    plotter.add_mesh(surface, color="red", label=f"Failure Surface {i+1}", opacity=1)

# Plot failure directions
for i, surface_cells in enumerate(failure_surfaces):
    surface = mesh.extract_cells(surface_cells)
    center = np.array(surface.center)  # Ensure center is a NumPy array
    direction = failure_directions[i]

    # Determine vector direction based on failure direction
    if direction == "Vertical":
        vector = np.array([0, 0, 1])
    elif direction == "Horizontal":
        vector = np.array([1, 0, 0])
    else:  # Oblique
        vector = np.array([1, 1, 1])

    # Normalize and scale the vector for visibility
    vector = vector / np.linalg.norm(vector) * 4

    plotter.add_arrows(center, vector, color="blue")

# Convert failure_magnitude to a NumPy array
failure_magnitude_np = failure_magnitude.get()

# Plot failure magnitude using a colormap
plotter.add_mesh(
    mesh, scalars=failure_magnitude_np, cmap="spectral", show_edges=True, opacity=0.1
)

# Add a color bar for failure magnitude
# plotter.add_scalar_bar("Failure Magnitude")

# Set camera position and show the plot
plotter.show_grid()
plotter.show()

# Plot causes of failure (optional 2D plot)
fig, ax = plt.subplots()

# Extract data for plotting
max_stress_values = [cause["max_stress"].get() for cause in failure_causes]
max_strain_values = [cause["max_strain"].get() for cause in failure_causes]
min_fos_values = [cause["min_fos"].get() for cause in failure_causes]

# Plot the data
ax.scatter(
    max_stress_values,
    max_strain_values,
    c=min_fos_values,
    cmap="coolwarm",
    s=100,
    edgecolors="black",
)

# Add labels and a color bar
ax.set_xlabel("Max Stress")
ax.set_ylabel("Max Strain")
ax.set_title("Causes of Failure")
cbar = plt.colorbar(ax.collections[0], ax=ax, label="Min FoS")

plt.show()

In [ ]:
def compute_resultant_direction_and_magnitude(
    failure_surfaces, mesh, failure_magnitude
):
    """
    Compute the resultant direction and magnitude of failure for each failure surface.

    Args:
        failure_surfaces (list): List of extracted failure surfaces.
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failure_magnitude (cp.ndarray): Magnitude of failure for each element.

    Returns:
        list: Resultant directions as normalized vectors.
        list: Resultant magnitudes for each failure surface.
    """
    print("Computing resultant direction and magnitude...")

    resultant_directions = []
    resultant_magnitudes = []

    failure_magnitude_np = failure_magnitude.get()  # Convert to NumPy array once

    for surface_cells in failure_surfaces:
        # Extract the cells for the current surface
        surface = mesh.extract_cells(surface_cells)
        surface_polydata = surface.extract_surface()

        # Compute normals for the surface if not already available
        surface_with_normals = surface_polydata.compute_normals(
            point_normals=True, cell_normals=False, inplace=False
        )
        normals = surface_with_normals.point_data["Normals"]

        # Convert to NumPy
        normals_np = np.array(normals)

        # Calculate the resultant vector for direction by averaging the normals
        resultant_vector = np.mean(normals_np, axis=0)
        resultant_vector /= np.linalg.norm(resultant_vector)  # Normalize

        # Compute the resultant magnitude as the sum of failure magnitudes for this surface
        surface_failure_magnitude = failure_magnitude_np[surface_cells]
        resultant_magnitude = np.sum(surface_failure_magnitude)

        resultant_directions.append(resultant_vector)
        resultant_magnitudes.append(resultant_magnitude)

    print("Resultant direction and magnitude computation completed.")
    return resultant_directions, resultant_magnitudes


def plot_resultant_directions_and_magnitudes(
    mesh, failure_surfaces, resultant_directions, resultant_magnitudes
):
    """
    Plot resultant directions and magnitudes on the mesh.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failure_surfaces (list): List of extracted failure surfaces.
        resultant_directions (list): List of resultant directions for each failure surface.
        resultant_magnitudes (list): Resultant magnitudes for each failure surface.
    """
    print("Plotting resultant directions and magnitudes...")

    plotter = pv.Plotter()
    plotter.add_mesh(mesh, show_edges=True, opacity=0.3, label="Mesh")

    for i, surface_cells in enumerate(failure_surfaces):
        # Extract the cells for the current surface
        surface = mesh.extract_cells(surface_cells)
        surface_polydata = surface.extract_surface()

        # Plot arrows representing resultant direction
        centers = surface_polydata.cell_centers().points
        resultant_vector = resultant_directions[i]

        # Scale the arrows by magnitude for visualization
        magnitude_scale = (
            resultant_magnitudes[i] / np.max(resultant_magnitudes)
            if np.max(resultant_magnitudes) > 0
            else 1.0
        )
        plotter.add_arrows(
            centers,
            np.tile(resultant_vector, (centers.shape[0], 1)),
            mag=0.1 * magnitude_scale,
            color="red",
        )

    # Add color bar for failure magnitude
    plotter.add_scalar_bar(title="Failure Magnitude", n_labels=5)

    # Show plot
    plotter.show()


# Example usage
failing_elements = identify_failing_elements(fos, threshold=1.0)
analysis_results_optimized = analyze_stress_strain_optimized(
    stresses, stresses, yield_stress=300e6, ultimate_stress=500e6
)
connected_components = connected_component_analysis(mesh, failing_elements)
failure_surfaces = extract_failure_surfaces(mesh, connected_components)
failure_causes = analyze_cause_of_failure(failure_surfaces, stresses, stresses, fos)
failure_directions = extract_failure_directions(failure_surfaces, mesh)
failure_magnitude = calculate_failure_magnitude(stresses, stresses, fos)

# print("Failure causes:", failure_causes)
# print("Failure directions:", failure_directions)
# print("Failure magnitudes:", failure_magnitude)

# Compute resultant directions and magnitudes
resultant_directions, resultant_magnitudes = compute_resultant_direction_and_magnitude(
    failure_surfaces, mesh, failure_magnitude
)

# Plotting the resultant directions and magnitudes
plot_resultant_directions_and_magnitudes(
    mesh, failure_surfaces, resultant_directions, resultant_magnitudes
)

In [ ]:
# Function to plot the original mesh
def plot_original_mesh(mesh):
    """
    Plot the original mesh before any failure analysis.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
    """
    plotter = pv.Plotter()
    plotter.add_mesh(
        mesh, show_edges=True, opacity=0.5, color="lightgrey", label="Original Mesh"
    )
    plotter.add_scalar_bar(title="Original Mesh")
    plotter.show()


# Function to compute resultant directions and magnitudes
def compute_resultant_direction_and_magnitude(
    failure_surfaces, mesh, failure_magnitude
):
    """
    Compute the resultant direction and magnitude of failure for each failure surface.

    Args:
        failure_surfaces (list): List of extracted failure surfaces.
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failure_magnitude (cp.ndarray): Magnitude of failure for each element.

    Returns:
        list: Resultant directions as normalized vectors.
        list: Resultant magnitudes for each failure surface.
    """
    resultant_directions = []
    resultant_magnitudes = []

    failure_magnitude_np = failure_magnitude.get()  # Convert to NumPy array once

    for surface_cells in failure_surfaces:
        # Extract the cells for the current surface
        surface = mesh.extract_cells(surface_cells)
        surface_polydata = surface.extract_surface()

        # Compute normals for the surface if not already available
        surface_with_normals = surface_polydata.compute_normals(
            point_normals=True, cell_normals=False, inplace=False
        )
        normals = surface_with_normals.point_data["Normals"]

        # Convert to NumPy
        normals_np = np.array(normals)

        # Calculate the resultant vector for direction by averaging the normals
        resultant_vector = np.mean(normals_np, axis=0)
        resultant_vector /= np.linalg.norm(resultant_vector)  # Normalize

        # Compute the resultant magnitude as the sum of failure magnitudes for this surface
        surface_failure_magnitude = failure_magnitude_np[surface_cells]
        resultant_magnitude = np.sum(surface_failure_magnitude)

        resultant_directions.append(resultant_vector)
        resultant_magnitudes.append(resultant_magnitude)

    return resultant_directions, resultant_magnitudes


# Function to plot the mesh after failure analysis
def plot_failure_results(
    mesh, failure_surfaces, resultant_directions, resultant_magnitudes
):
    """
    Plot resultant directions and magnitudes on the mesh after failure analysis.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failure_surfaces (list): List of extracted failure surfaces.
        resultant_directions (list): List of resultant directions for each failure surface.
        resultant_magnitudes (list): Resultant magnitudes for each failure surface.
    """
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, show_edges=True, opacity=0.3, label="Mesh")

    for i, surface_cells in enumerate(failure_surfaces):
        # Extract the cells for the current surface
        surface = mesh.extract_cells(surface_cells)
        surface_polydata = surface.extract_surface()

        # Plot arrows representing resultant direction
        centers = surface_polydata.cell_centers().points
        resultant_vector = resultant_directions[i]

        # Scale the arrows by magnitude for visualization
        magnitude_scale = (
            resultant_magnitudes[i] / np.max(resultant_magnitudes)
            if np.max(resultant_magnitudes) > 0
            else 1.0
        )
        plotter.add_arrows(
            centers,
            np.tile(resultant_vector, (centers.shape[0], 1)),
            mag=0.1 * magnitude_scale,
            color="red",
        )

    # Add color bar for failure magnitude
    plotter.add_scalar_bar(title="Failure Magnitude", n_labels=5)
    plotter.show()


# Function to highlight failing cells on the mesh
def plot_failing_cells(mesh, failing_elements):
    """
    Plot the failing cells on the mesh after failure analysis.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failing_elements (cp.ndarray): Indices of elements with FoS ≤ threshold.
    """
    plotter = pv.Plotter()
    plotter.add_mesh(
        mesh, show_edges=True, opacity=0.3, color="lightgrey", label="Mesh"
    )

    # Convert failing elements to NumPy array for plotting
    failing_elements_np = failing_elements.get()

    # Extract and highlight failing cells
    failing_cells = mesh.extract_cells(failing_elements_np)
    plotter.add_mesh(failing_cells, color="red", label="Failing Cells")

    # Show plot
    plotter.show()


# Example usage
failing_elements = identify_failing_elements(fos, threshold=1.0)
analysis_results_optimized = analyze_stress_strain_optimized(
    stresses, stresses, yield_stress=300e6, ultimate_stress=500e6
)
connected_components = connected_component_analysis(mesh, failing_elements)
failure_surfaces = extract_failure_surfaces(mesh, connected_components)
failure_magnitude = calculate_failure_magnitude(stresses, stresses, fos)

# Compute resultant directions and magnitudes
resultant_directions, resultant_magnitudes = compute_resultant_direction_and_magnitude(
    failure_surfaces, mesh, failure_magnitude
)

# Plotting the original mesh
plot_original_mesh(mesh)

# Plotting the failure results
plot_failure_results(mesh, failure_surfaces, resultant_directions, resultant_magnitudes)

# Highlighting failing cells after failure
plot_failing_cells(mesh, failing_elements)

In [ ]:
# Function to plot mesh with failure features
def plot_failure_features(
    mesh, failing_elements, failure_surfaces, resultant_directions, resultant_magnitudes
):
    """
    Plot the mesh with failure features, similar to the provided image.

    Args:
        mesh (pv.UnstructuredGrid): The mesh of the model.
        failing_elements (cp.ndarray): Indices of elements with FoS ≤ threshold.
        failure_surfaces (list): List of extracted failure surfaces.
        resultant_directions (list): List of resultant directions for each failure surface.
        resultant_magnitudes (list): Resultant magnitudes for each failure surface.
    """
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, show_edges=True, opacity=0.3, label="Mesh")

    # Highlight failing elements
    failing_elements_np = failing_elements.get()
    failing_cells = mesh.extract_cells(failing_elements_np)
    plotter.add_mesh(failing_cells, color="red", label="Failing Cells")

    # Plot failure directions and magnitudes
    for i, surface_cells in enumerate(failure_surfaces):
        surface = mesh.extract_cells(surface_cells)
        surface_polydata = surface.extract_surface()

        # Plot arrows representing resultant direction
        centers = surface_polydata.cell_centers().points
        resultant_vector = resultant_directions[i]

        # Scale the arrows by magnitude for visualization
        magnitude_scale = (
            resultant_magnitudes[i] / np.max(resultant_magnitudes)
            if np.max(resultant_magnitudes) > 0
            else 1.0
        )
        plotter.add_arrows(
            centers,
            np.tile(resultant_vector, (centers.shape[0], 1)),
            mag=0.1 * magnitude_scale,
            color="blue",
        )

    # Annotate key failure features (example positions, adjust as needed)
    plotter.add_text("Crown", position=(0.1, 0.8), color="black", font_size=12)
    plotter.add_text("Minor Scarp", position=(0.2, 0.7), color="black", font_size=12)
    plotter.add_text("Main Body", position=(0.4, 0.5), color="black", font_size=12)
    plotter.add_text("Toe of Rupture", position=(0.6, 0.3), color="black", font_size=12)

    # Add color bar for failure magnitude
    plotter.add_scalar_bar(title="Failure Magnitude", n_labels=5)

    # Show plot
    plotter.show()


# Example usage
failing_elements = identify_failing_elements(fos, threshold=1.0)
analysis_results_optimized = analyze_stress_strain_optimized(
    stresses, stresses, yield_stress=300e6, ultimate_stress=500e6
)
connected_components = connected_component_analysis(mesh, failing_elements)
failure_surfaces = extract_failure_surfaces(mesh, connected_components)
failure_magnitude = calculate_failure_magnitude(stresses, stresses, fos)

# Compute resultant directions and magnitudes
resultant_directions, resultant_magnitudes = compute_resultant_direction_and_magnitude(
    failure_surfaces, mesh, failure_magnitude
)

# Plotting the mesh with failure features
plot_failure_features(
    mesh, failing_elements, failure_surfaces, resultant_directions, resultant_magnitudes
)